<a href="https://colab.research.google.com/github/ChristopherMajor/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Christopher_Major_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
top10

Index(['FLUSHING-NORTH', 'UPPER EAST SIDE (59-79)', 'UPPER EAST SIDE (79-96)',
       'BEDFORD STUYVESANT', 'BOROUGH PARK', 'UPPER WEST SIDE (59-79)',
       'GRAMERCY', 'ASTORIA', 'FOREST HILLS', 'UPPER WEST SIDE (79-96)'],
      dtype='object')

In [0]:
df['SALE_PRICE'].describe()

count    2.304000e+04
mean     1.328133e+06
std      1.025394e+07
min      0.000000e+00
25%      0.000000e+00
50%      4.550000e+05
75%      8.750000e+05
max      8.500000e+08
Name: SALE_PRICE, dtype: float64

In [0]:
#subsetting data
condition = ((df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS') & 
            (df['SALE_PRICE'] > 100000) &
            (df['SALE_PRICE'] < 2000000))

In [0]:
df= df[condition]

In [0]:
#checking it worked
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [0]:
#setting up for the split
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('01/04/2019')


In [0]:
df.SALE_DATE < cutoff

44        True
61        True
78        True
108       True
111       True
         ...  
23029    False
23031    False
23032    False
23033    False
23035    False
Name: SALE_DATE, Length: 3151, dtype: bool

In [0]:
#splitted
train = df[df.SALE_DATE < cutoff]
test = df[df.SALE_DATE >= cutoff]

In [0]:
#analysing dataset before OneHot
test['NEIGHBORHOOD'].describe()

count      3088
unique        6
top       OTHER
freq       2931
Name: NEIGHBORHOOD, dtype: object

In [0]:
train.describe(include='all')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
count,63,63,63,63,63.000000,63.000000,0.0,63,63,0,63.000000,63.0,63.000000,63.000000,63,63.000000,63.000000,63.0,63,6.300000e+01,63
unique,4,2,1,1,NaN,NaN,NaN,7,63,0,NaN,NaN,NaN,NaN,40,NaN,NaN,NaN,7,NaN,3
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,NaN,NaN,NaN,A1,17 RATHBUN AVENUE,NaN,NaN,NaN,NaN,NaN,"4,000",NaN,NaN,NaN,A1,NaN,2019-01-03 00:00:00
freq,35,59,63,63,NaN,NaN,NaN,19,1,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,19,NaN,40
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-03 00:00:00
mean,NaN,NaN,NaN,NaN,7063.428571,120.460317,NaN,NaN,NaN,NaN,11123.619048,1.0,0.031746,1.031746,NaN,1689.539683,1947.539683,1.0,NaN,6.908303e+05,NaN
std,NaN,NaN,NaN,NaN,3927.612743,276.537976,NaN,NaN,NaN,NaN,441.606431,0.0,0.176731,0.176731,NaN,568.764502,27.846251,0.0,NaN,2.985463e+05,NaN
min,NaN,NaN,NaN,NaN,377.000000,1.000000,NaN,NaN,NaN,NaN,10302.000000,1.0,0.000000,1.000000,NaN,736.000000,1899.000000,1.0,NaN,1.250000e+05,NaN
25%,NaN,NaN,NaN,NaN,4578.500000,25.000000,NaN,NaN,NaN,NaN,11205.000000,1.0,0.000000,1.000000,NaN,1310.500000,1925.000000,1.0,NaN,4.800000e+05,NaN


In [0]:
train.head(10)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,2019-01-02
120,3,OTHER,01 ONE FAMILY DWELLINGS,1,6806,72,NaN,A1,1747 EAST 23RD STREET,NaN,11229.0,1.0,0.0,1.0,"4,000",1932.0,1930.0,1,A1,1150000,2019-01-02
121,3,OTHER,01 ONE FAMILY DWELLINGS,1,6761,42,NaN,A1,1582 EAST 15TH STREET,NaN,11230.0,1.0,0.0,1.0,"2,000",1722.0,1920.0,1,A1,836500,2019-01-02
132,4,OTHER,01 ONE FAMILY DWELLINGS,1,7378,4,NaN,A1,201-08 50TH AVENUE,NaN,11364.0,1.0,0.0,1.0,"3,000",1396.0,1930.0,1,A1,955000,2019-01-02
137,4,OTHER,01 ONE FAMILY DWELLINGS,1,2882,16,NaN,A5,85-11 57 ROAD,NaN,11373.0,1.0,0.0,1.0,"1,800",1260.0,1935.0,1,A5,820000,2019-01-02
141,4,FLUSHING-NORTH,01 ONE FAMILY DWELLINGS,1,7407,50,NaN,A0,53-19 198TH STREET,NaN,11365.0,1.0,0.0,1.0,"4,000",1440.0,1945.0,1,A0,980000,2019-01-02


In [0]:
#prepping features for one hot encoding and setting target
target= 'SALE_PRICE'

# high_card= ['BLOCK', 'LOT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'ZIP_CODE',
#             'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
#             'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_DATE', 'NEIGHBORHOOD']

pointless= ['TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_CATEGORY',
            'TAX_CLASS_AT_PRESENT', 'APARTMENT_NUMBER', 'EASE-MENT', 'ADDRESS']

features = train.columns.drop([target]+ pointless)

In [0]:

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]


In [0]:
#here comes the OneHot!
import category_encoders as ce 
encoder = ce.OneHotEncoder(cols = ('BOROUGH','NEIGHBORHOOD'), use_cat_names=True)

#fit encoder to train data
X_train = encoder.fit_transform(X_train)

In [0]:
X_train

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
44,1,0,0,0,1,0,5495,801,A9,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,A9,2019-01-01
61,0,1,0,0,1,0,7918,72,A1,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,A1,2019-01-01
78,0,0,1,0,1,0,4210,19,A1,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,A1,2019-01-02
108,1,0,0,0,1,0,5212,69,A1,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,A1,2019-01-02
111,1,0,0,0,1,0,7930,121,A5,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,A5,2019-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,0,0,0,1,1,0,3736,26,A5,10306.0,1.0,0.0,1.0,"2,325",1888.0,2017.0,A5,2019-01-03
441,0,0,0,1,1,0,1016,29,A1,10302.0,1.0,0.0,1.0,"2,527",1428.0,1920.0,A1,2019-01-03
450,0,0,0,1,1,0,3246,19,A9,10305.0,1.0,0.0,1.0,"2,058",1230.0,1960.0,A9,2019-01-03
455,0,0,0,1,1,0,1984,106,A3,10314.0,1.0,0.0,1.0,"6,935",3050.0,1985.0,A3,2019-01-03


In [0]:
#one hot test data
X_test = encoder.transform(X_test)
X_test

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
513,0,0,1,0,1,0,4735,86,A5,10469.0,1.0,0.0,1.0,"2,217",1620.0,1930.0,A5,2019-01-04
515,0,0,1,0,1,0,4634,70,A5,10469.0,1.0,0.0,1.0,"2,000",1727.0,1935.0,A5,2019-01-04
523,0,0,1,0,1,0,5761,458,A9,10463.0,1.0,0.0,1.0,"3,655",1748.0,1915.0,A9,2019-01-04
549,1,0,0,0,0,0,1822,59,A4,11216.0,1.0,0.0,1.0,"1,917",2877.0,1910.0,A4,2019-01-04
568,1,0,0,0,1,0,3773,134,A5,11207.0,1.0,0.0,1.0,"1,800",1134.0,1992.0,A5,2019-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,0,1,0,0,1,0,13215,3,A2,11422.0,1.0,0.0,1.0,"3,300",1478.0,1925.0,A2,2019-04-30
23031,0,1,0,0,1,0,11612,73,A1,11420.0,1.0,0.0,1.0,"2,400",1280.0,1930.0,A1,2019-04-30
23032,0,1,0,0,1,0,11808,50,A0,11420.0,1.0,0.0,1.0,"4,000",1333.0,1945.0,A0,2019-04-30
23033,0,1,0,0,1,0,12295,23,A1,11434.0,1.0,0.0,1.0,"2,500",1020.0,1935.0,A1,2019-04-30


In [0]:
#import and instantiate KBest on top 10 features
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k=7)

In [0]:
X_train['LAND_SQUARE_FEET'] = (
    X_train['LAND_SQUARE_FEET']
    #.str.replace('$','')
    #.str.replace('-','')
    .str.replace(',','')
    .astype(int)
)
X_train['LAND_SQUARE_FEET'].type()

In [0]:
X_test['LAND_SQUARE_FEET'] = (
    X_test['LAND_SQUARE_FEET']
    #.str.replace('$','')
    #.str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
X_test.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
513,0,0,1,0,1,0,4735,86,A5,10469.0,1.0,0.0,1.0,2217,1620.0,1930.0,A5,2019-01-04
515,0,0,1,0,1,0,4634,70,A5,10469.0,1.0,0.0,1.0,2000,1727.0,1935.0,A5,2019-01-04
523,0,0,1,0,1,0,5761,458,A9,10463.0,1.0,0.0,1.0,3655,1748.0,1915.0,A9,2019-01-04
549,1,0,0,0,0,0,1822,59,A4,11216.0,1.0,0.0,1.0,1917,2877.0,1910.0,A4,2019-01-04
568,1,0,0,0,1,0,3773,134,A5,11207.0,1.0,0.0,1.0,1800,1134.0,1992.0,A5,2019-01-04


In [0]:
X_test=X_test.drop(labels=['BUILDING_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_PRESENT'], axis=1)
X_test

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,SALE_DATE
513,0,0,1,0,1,0,4735,86,10469.0,1.0,0.0,1.0,2217,1620.0,1930.0,2019-01-04
515,0,0,1,0,1,0,4634,70,10469.0,1.0,0.0,1.0,2000,1727.0,1935.0,2019-01-04
523,0,0,1,0,1,0,5761,458,10463.0,1.0,0.0,1.0,3655,1748.0,1915.0,2019-01-04
549,1,0,0,0,0,0,1822,59,11216.0,1.0,0.0,1.0,1917,2877.0,1910.0,2019-01-04
568,1,0,0,0,1,0,3773,134,11207.0,1.0,0.0,1.0,1800,1134.0,1992.0,2019-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,0,1,0,0,1,0,13215,3,11422.0,1.0,0.0,1.0,3300,1478.0,1925.0,2019-04-30
23031,0,1,0,0,1,0,11612,73,11420.0,1.0,0.0,1.0,2400,1280.0,1930.0,2019-04-30
23032,0,1,0,0,1,0,11808,50,11420.0,1.0,0.0,1.0,4000,1333.0,1945.0,2019-04-30
23033,0,1,0,0,1,0,12295,23,11434.0,1.0,0.0,1.0,2500,1020.0,1935.0,2019-04-30


In [0]:
#X_train=X_train.drop(labels=['BUILDING_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'EASE-MENT'], axis=1)
X_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,SALE_DATE
44,1,0,0,0,1,0,5495,801,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0,2019-01-01
61,0,1,0,0,1,0,7918,72,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0,2019-01-01
78,0,0,1,0,1,0,4210,19,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0,2019-01-02
108,1,0,0,0,1,0,5212,69,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0,2019-01-02
111,1,0,0,0,1,0,7930,121,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0,2019-01-02


In [0]:
X_train=X_train.drop('SALE_DATE', axis=1)

In [0]:
X_test=X_test.drop('SALE_DATE', axis=1)

In [0]:
X_test.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT
513,0,0,1,0,1,0,4735,86,10469.0,1.0,0.0,1.0,2217,1620.0,1930.0
515,0,0,1,0,1,0,4634,70,10469.0,1.0,0.0,1.0,2000,1727.0,1935.0
523,0,0,1,0,1,0,5761,458,10463.0,1.0,0.0,1.0,3655,1748.0,1915.0
549,1,0,0,0,0,0,1822,59,11216.0,1.0,0.0,1.0,1917,2877.0,1910.0
568,1,0,0,0,1,0,3773,134,11207.0,1.0,0.0,1.0,1800,1134.0,1992.0


In [0]:
X_train_selected= selector.fit_transform(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [0]:
X_train_selected.shape

(63, 7)

In [0]:
X_test_selected= selector.transform(X_test)

In [0]:
X_test_selected.shape

(3088, 7)

In [0]:
selected_mask= selector.get_support()
all_names=X_train.columns
selected_names = all_names[selected_mask]
unselected_names= all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
  print(name)


Features selected:
BOROUGH_3
BOROUGH_4
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
BLOCK
LAND_SQUARE_FEET
GROSS_SQUARE_FEET
